1. Import libraries and create a Spark session:

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("ProductCategoryAssignment").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

2. Create DataFrames with product, category, and their relationship data:

In [3]:
products_data = [
    (1, "Product A"),
    (2, "Product B"),
    (3, "Product C"),
    (4, "Product D"),
]
products_df = spark.createDataFrame(products_data, ["product_id", "product_name"])

categories_data = [
    (1, "Category X"),
    (2, "Category Y"),
    (3, "Category Z"),
]
categories_df = spark.createDataFrame(categories_data, ["category_id", "category_name"])

product_category_data = [
    (1, 1),
    (2, 2),
    (3, 3),
    (1, 2),
    (4, None),
]
product_category_df = spark.createDataFrame(product_category_data, ["product_id", "category_id"])

3. Define a method to get "Product Name - Category Name" pairs and a list of products without categories:

In [6]:
def get_product_category_pairs(products_df, categories_df, product_category_df):
    # Join products and relationships
    product_category_joined = products_df.join(product_category_df, "product_id", "left")
    
    # Join with categories
    result_df = product_category_joined.join(categories_df, "category_id", "left")
    
    # "Product Name - Category Name" pairs
    product_category_pairs = result_df.select("product_name", "category_name")
    
    # Products without categories
    products_without_category = product_category_pairs.filter(col("category_name").isNull()).select("product_name")
    
    return product_category_pairs, products_without_category

4. Get and display the results:

In [7]:
product_category_pairs, products_without_category = get_product_category_pairs(products_df, categories_df, product_category_df)

product_category_pairs.show()
products_without_category.show()

+------------+-------------+
|product_name|category_name|
+------------+-------------+
|   Product A|   Category Y|
|   Product A|   Category X|
|   Product B|   Category Y|
|   Product C|   Category Z|
|   Product D|         NULL|
+------------+-------------+



+------------+
|product_name|
+------------+
|   Product D|
+------------+

